# Home Credit Default Risk
Home Credit is an international non-bank, consumer finance group. The company operates in 14 countries and focuses on lending primarily to people with little or no credit history.

*Credit* is the trust which allows one party to provide money or resources to another party where that second party does not reimburse the first party immediately (*thereby generating a debt*), but instead promises either to repay or return those resources at a later date. A *credit bureau* is a collection agency that gathers account information from various creditors.

This notebook aims to be able to **predict how capable an applicant is of repaying a loan**, using datasets, provided by Home Credit.

[Link](https://www.kaggle.com/c/home-credit-default-risk/data) to the datasets.

## Attributes

A **cash loan** is the lending of money by one or more individuals, organizations, or other entities to other individuals, organizations etc. The recipient (i.e. the borrower) incurs a debt, and is usually liable to pay interest on that debt until it is repaid, and also to repay the principal amount borrowed.  
**Revolving credit/loan** is a type of credit that does not have a fixed number of payments, in contrast to installment credit. Credit cards are an example of revolving credit used by consumers.


In [67]:
# Manipulating data
import numpy as np
import pandas as pd

# Managing files
import os

## The different datasets

* **application_train.csv** and **application_test.csv**
  * The main table in two files. The train set contains __labels/targets__ (0: the loan was *repaid* or 1: the loan was *not* repaid), and the test set does not.
  * Static data for all applications. One row represents one loan.  
  
  
* **bureau.csv**
  * All client's previous credits provided by other financial institutions reported by the Credit Bureau (loans).
  * For every loan, there are as many rows as number of credits the client had in Credit Bureau before the application date. In other words, each previous credit has its own row in bureau, but one loan in the application data can have multiple previous credits.
  
  
* **bureau_balance.csv**
  * Monthly balances of previous credits in Credit Bureau (**bureau.csv**).
  * This table has one row for each month of history of every previous credit reported to Credit Bureau - the table has (*# loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits*) rows. In other words, each row is one month of a previous credit, and a single previous credit can have multiple rows, *one for each month* of the credit length.
  
  
* **POS_CASH_balance.csv**
  * Monthly balance snapshots/data of previous **POS (point of sales) and cash loans** that the applicant had with Home Credit.
  * This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (*# loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits*) rows.
  
  
* **credit_card_balance.csv**
  * Monthly balance snapshots of previous **credit cards** that the applicant has with Home Credit.
  * This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample - the table has (*# loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card*) rows. In other words, each row is one month of a credit card balance, and a single credit card can have many rows.
  
  
* **previous_application.csv**
  * All previous applications for Home Credit loans of *clients who have loans* in our sample.
  * There is one row for each previous application related to loans in our data sample. Each current loan in the application data can have multiple previous loans. Each previous application has one row and is identified by the feature SK_ID_PREV. 
  
  
* **installments_payments.csv** (*A sum of money paid in small parts in a fixed period of time or a single payment within a staged payment plan of a loan*)
  * Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
  * There is a) one row for **every payment that was made** plus b) one row each for **missed payment**.
  * One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.
  
  
* **HomeCredit_columns_description.csv**
  * This file contains descriptions for the columns in the various data files.

<img src="misc/datasets.png" title="The datasets." />

In [68]:
print(os.listdir("datasets/"))

['credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'sample_submission.csv', 'bureau.csv', 'previous_application.csv', 'POS_CASH_balance.csv', '.~lock.HomeCredit_columns_description.csv#', 'application_train.csv', 'application_test.csv', 'bureau_balance.csv']


In [69]:
train_set = pd.read_csv('datasets/application_train.csv')
test_set = pd.read_csv('datasets/application_test.csv')

## Exploring the application train and test sets

Exploring the data to learn more about the data we have available. We dive into the application_train and application_test set to get a solid grasp of the main data and not branch out too widely into the other sets before we have solid grounds to work from.

In [70]:
print("The training set contains the following number of rows and columns", train_set.shape)

The training set contains the following number of rows and columns (307511, 122)


In [71]:
train_set.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
list(train_set)

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_A

In [73]:
train_set.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
print("The test set contains the following number of rows and columns", test_set.shape, "- missing the TARGET column")

The test set contains the following number of rows and columns (48744, 121) - missing the TARGET column


### How many missing values are we dealing with?

It is important to lay out an overview over the amount of missing values in the dataset. We need to make a decision later on what to do with them, if anything at all.

In [75]:
# Function to calculate missing values by column# 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [76]:
missing_values_table(train_set)

Your selected dataframe has 122 columns.
There are 67 columns that have missing values.


,Missing Values,% of Total Values
COMMONAREA_MEDI,214865,69.9
COMMONAREA_AVG,214865,69.9
COMMONAREA_MODE,214865,69.9
NONLIVINGAPARTMENTS_MEDI,213514,69.4
NONLIVINGAPARTMENTS_MODE,213514,69.4
NONLIVINGAPARTMENTS_AVG,213514,69.4
FONDKAPREMONT_MODE,210295,68.4
LIVINGAPARTMENTS_MODE,210199,68.4
LIVINGAPARTMENTS_MEDI,210199,68.4
LIVINGAPARTMENTS_AVG,210199,68.4


If you look above the last 18 columns, you'll see that 49 columns has around 50% to almost 70% of their data set to null, or as empty rows.

### Converting categorical attributes to numerical ones

Generally, machine learning attributes don't work on categorical attributes that aren't numbers. Therefore, we need to convert these into numerical attributes using One-hot encoding to ensure that these will work in our model. That is, to create a new column for each categorical variable. The label encoding method produces a cleaner results with less dimensions, but assigns arbitrary values to each categorical variable, maybe based on relative value of the feature, which creates another set of problems where these categories might be weighted differently when they should'nt be.

The object type is what needs to be converted. We know these consists of strings as we loaded the data from a .csv fil

In [77]:
train_set.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

In [78]:
# Return number of unique elements in the 16 objects
train_set.select_dtypes('object').apply(pd.Series.nunique, axis=0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

We'll use a label encoder for the data which has at most 2 categorical variables. This will keep the number of columns lower, while at the same time not risking arbitrary categorical values being weighted differently.

In [79]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in train_set:
    if train_set[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train_set[col].unique())) <= 2:
            # Train on the training data
            le.fit(train_set[col])
            # Transform both training and testing data
            train_set[col] = le.transform(train_set[col])
            test_set[col] = le.transform(test_set[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


One-hot encoding for the rest of the columns with categorical variables > 2:

In [80]:
train_columns_before = train_set.shape[1]
test_columns_before = test_set.shape[1]

In [81]:
train_set = pd.get_dummies(train_set)
test_set = pd.get_dummies(test_set)

In [82]:
print("Training set columns", train_columns_before,"->", train_set.shape[1])
print("Test set columns", test_columns_before,"->",test_set.shape[1])

Training set columns 122 -> 243
Test set columns 121 -> 239


It's important that we make sure the training set and the test set has the same amount, and type, of columns. We are training on a particular type of data, and the format of this data cannot vary, only its content. How else will we be able to measure how good our model is?

We discard whatever columns are not present in both sets. 

In [83]:
# Saving the labels for the training set
train_target_values = train_set["TARGET"]

# Takes the intersection (inner) of the columns (axis=1) and apply them to both
train_set, test_set = train_set.align(test_set, join='inner', axis=1)

# Reintroduce labels to the training set
train_set["TARGET"] = train_target_values

In [84]:
print("Num of columns in training set:", train_set.shape[1])
print("Num of columns in test set:", test_set.shape[1])

Num of columns in training set: 240
Num of columns in test set: 239


## Diving into the data

#### Detecting anomalies
Anomalies are outliers that can be explained by extreme measurements, errors while measured or mis-typing/error when parsing the data. Either way, they need to be dealt with.

In [90]:
# Client's age in days at the time of application
train_set['DAYS_BIRTH'].describe() # Statistics about a column, excluding NaN values.

count    307511.000000
mean     -16036.995067
std        4363.988632
min      -25229.000000
25%      -19682.000000
50%      -15750.000000
75%      -12413.000000
max       -7489.000000
Name: DAYS_BIRTH, dtype: float64

In [91]:
# in years
(train_set['DAYS_BIRTH'] / -365).describe()

count    307511.000000
mean         43.936973
std          11.956133
min          20.517808
25%          34.008219
50%          43.150685
75%          53.923288
max          69.120548
Name: DAYS_BIRTH, dtype: float64

### Repaid vs not repaid

Repaid loans (0) compared to not repaid loans (1) shows us that, *there are far more repaid loans than not repaid loans*. Most machine learning algorithms work best when the number of classes are roughly equal. This is called the *class imbalance problem*. We want to emphasize to the machine learning algorithm that it's important to detect the few cases where a loan is likely not to be repaid, rather than focusing only on high accuracy.

In [85]:
train_set["TARGET"].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [86]:
num_of_not_repaid = train_set["TARGET"].sum()
not_repaid_percentage = num_of_not_repaid / train_set["TARGET"].count() * 100
print("%.2f" % not_repaid_percentage, "% did not repay their loans")

8.07 % did not repay their loans


#### (Put on halt for now) Training a temporary Binary Classifier on income and credit alone

We create a temporary set of the total amount of income and credit and explore what kind of predictions we can get from it.

In [87]:
from sklearn.linear_model import SGDClassifier

income_and_credit = train_set[["AMT_INCOME_TOTAL","AMT_CREDIT"]]
income_and_credit_labeled = train_set["TARGET"]
income_and_credit.head()

,AMT_INCOME_TOTAL,AMT_CREDIT
0,202500.0,406597.5
1,270000.0,1293502.5
2,67500.0,135000.0
3,135000.0,312682.5
4,121500.0,513000.0


In [88]:
# using random state makes the results reproducible
sgd_clf = SGDClassifier(random_state=30)
sgd_clf.fit(income_and_credit, income_and_credit_labeled)

/home/adr/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=30, shuffle=True,
       tol=None, verbose=0, warm_start=False)

Predicting the first and second row, which we know are 1 and 0 respectively

### Crosstab

Inspecting the relation between target and gender (exlcuding "XNA").

In [89]:
train_set.CODE_GENDER.unique()

AttributeError: 'DataFrame' object has no attribute 'CODE_GENDER'

In [ ]:
pd.crosstab(train_set["TARGET"], train_set["CODE_GENDER"], margins=True)

There are 4 instances in the training set missing the gender data. We have to decide whether to fill those in, keep the data as is or discard them completely. In a crosstab though, the four instances won't produce valuable output.

In [ ]:
def toPercentages(self):
  return self/float(self[-1])

mf_train = train_set[train_set.CODE_GENDER != 'XNA']

In [ ]:
pd.crosstab(mf_train["TARGET"], mf_train["CODE_GENDER"], margins=True).apply(toPercentages, axis=0)

According to the gender data alone, it tells us that, in the training set, 10% of males don't repay their loan, and 7% of females don't repay their loans. Also, take into consideration there are almost twice the amount of data on females compared to males.

Another interesting comparison is the relation between if the applicant owns a car, or a house or flat.

In [ ]:
pd.crosstab(train_set["TARGET"], train_set["FLAG_OWN_CAR"], margins=True).apply(toPercentages, axis=0)

It doesn't seem that FLAG_OWN_CAR alone is gonna tell us anything interesting on its own. Maybe this column is additional information that won't give us any better predictions. We will continue on and revisit this later when we look closer into feature selection and feature reduction.

In [ ]:
pd.crosstab(train_set["TARGET"], train_set["FLAG_OWN_REALTY"], margins=True).apply(toPercentages, axis=0)

Again, no strong correlation between owning a house or a flat and repaying loans, according to the training set. 